<a href="https://colab.research.google.com/github/yang1325/AI_selfpraticenote/blob/main/tensorflow_covid/tensorflow_Coviddata_splitted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#載入所需模組
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow import keras
import numpy as np
import pandas as pd
from numpy import load
from numpy import save
import random
import os
import gc
import time

In [9]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [10]:
cnn_model = models.Sequential()
cnn_model.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(299,299, 1)))
cnn_model.add(layers.MaxPooling2D((3, 3)))
cnn_model.add(layers.Conv2D(256, (3, 3), activation='relu'))
cnn_model.add(layers.MaxPooling2D((3, 3)))
cnn_model.add(layers.Conv2D(512, (3, 3), activation='relu'))
cnn_model.add(layers.MaxPooling2D((3, 3)))
cnn_model.add(layers.Conv2D(1024, (3, 3), activation='relu'))
cnn_model.add(layers.MaxPooling2D((2, 2)))
#cnn_model.add(layers.Conv2D(1024, (3, 3), activation='relu'))
#cnn_model.add(layers.MaxPooling2D((3, 3)))
cnn_model.add(layers.Flatten())
cnn_model.add(tf.keras.layers.Dropout(0.3))
cnn_model.add(layers.Dense(512, activation='relu'))
cnn_model.add(layers.Dense(512, activation='relu'))
cnn_model.add(layers.Dense(4, activation="softmax"))
cnn_model.summary()
gc.collect()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 297, 297, 128)     1280      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 99, 99, 128)      0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 97, 97, 256)       295168    
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 32, 32, 256)      0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 30, 30, 512)       1180160   
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 10, 10, 512)     

643

In [11]:
path = "/content/gdrive/My Drive/splited_data"
checkpoint_path = "/content/gdrive/My Drive/trained/model(1).ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
        save_weights_only=True
        ,verbose=1)

cnn_model.compile(optimizer="Adam",
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

cnn_model.load_weights(checkpoint_path)

In [12]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
def loss(model, x, y, training):
  y_ = model(x, training=training)
  return loss_object(y_true=y, y_pred=y_)
def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets, training=True)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [13]:
for k in range(100):
  for j in range(10):
    print("now is %d,%d"%(k,j))
    data = load(path + "/%d"%(j) + ".npy")
    n = np.shape(data)[0]
    label = load(path + "/%dy"%(j) + ".npy")
    z = np.apply_along_axis(np.argmax,1,label)
    t = tf.convert_to_tensor(data)
    del data
    gc.collect()
    if(j==9):
      print(cnn_model.evaluate(t, label, verbose=2))
    else:
      #cnn_model.fit(data, label, epochs=1,batch_size = 1,callbacks=[cp_callback])
      epoch_loss_avg = tf.keras.metrics.Mean()
      epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
      index = np.array(random.sample(range(0,n),n))
      tem = 0
      for i in index:
        tem +=  1
        if(tem%100 == 0):
          current_time = time.strftime("%H:%M:%S", time.localtime())
          print(tem,",",current_time)
        loss_value, grads = grad(cnn_model, t[i:(i+1)], z[i:(i+1)])
        epoch_loss_avg.update_state(loss_value)
        epoch_accuracy.update_state(z[i:(i+1)], cnn_model(t[i:(i+1)], training=True))
      print()
      print("Epoch {},{}: Loss: {:.3f}, Accuracy: {:.3%}".format(k,j,
                 epoch_loss_avg.result(),
                 epoch_accuracy.result()))

    del t
    del label
    gc.collect()

串流輸出內容已截斷至最後 5000 行。
1400 , 07:50:47
1500 , 07:50:50
1600 , 07:50:53
1700 , 07:50:56
1800 , 07:50:59
1900 , 07:51:02
2000 , 07:51:05
2100 , 07:51:08

Epoch 12,2: Loss: 0.487, Accuracy: 81.002%
now is 12,3
100 , 07:51:24
200 , 07:51:27
300 , 07:51:30
400 , 07:51:33
500 , 07:51:36
600 , 07:51:39
700 , 07:51:42
800 , 07:51:45
900 , 07:51:48
1000 , 07:51:51
1100 , 07:51:54
1200 , 07:51:57
1300 , 07:52:00
1400 , 07:52:03
1500 , 07:52:06
1600 , 07:52:09
1700 , 07:52:12
1800 , 07:52:15
1900 , 07:52:18
2000 , 07:52:20
2100 , 07:52:23

Epoch 12,3: Loss: 0.511, Accuracy: 80.340%
now is 12,4
100 , 07:52:39
200 , 07:52:42
300 , 07:52:45
400 , 07:52:48
500 , 07:52:51
600 , 07:52:54
700 , 07:52:57
800 , 07:53:00
900 , 07:53:03
1000 , 07:53:06
1100 , 07:53:09
1200 , 07:53:12
1300 , 07:53:14
1400 , 07:53:17
1500 , 07:53:20
1600 , 07:53:23
1700 , 07:53:26
1800 , 07:53:29
1900 , 07:53:32
2000 , 07:53:35
2100 , 07:53:38

Epoch 12,4: Loss: 0.507, Accuracy: 80.435%
now is 12,5
100 , 07:53:54
200 , 07:53:57

KeyboardInterrupt: ignored